# Análise de todos os dados do Traefik

In [159]:
import pandas as pd

df = pd.read_csv('traefik.csv', sep=',')

### Existe algum padrão de horário?

In [160]:
df['data1'] = pd.to_datetime(df.data1, format='%Y-%m-%d %H:%M:%S')

In [164]:
hours = pd.DataFrame()
hours = pd.DataFrame(columns=['Horário', 'Qnt. de Acessos'])

In [165]:
for i in range(24):
    hours.loc[i]=[ i, df.data1[df['data1'].dt.hour == i].count()]

In [166]:
hours

,Horário,Qnt. de Acessos
0,0,3074
1,1,2368
2,2,2197
3,3,2031
4,4,2233
5,5,2428
6,6,2857
7,7,12626
8,8,43302
9,9,40530


In [167]:
dateAcess = df.groupby([df['data1'].dt.month, df['data1'].dt.year,]).count().sort_index()
dateAcess.rename(columns={'status': 'Acessos'}, inplace=True)
dateAcess = dateAcess.Acessos

In [168]:
dateAcess = dateAcess.rename_axis(['Mês', 'Ano']).reset_index()

In [169]:
dateAcess

,Mês,Ano,Acessos
0,1,2022,37149
1,2,2022,25189
2,3,2022,59927
3,4,2022,50877
4,5,2022,77551
5,6,2022,112896
6,7,2022,42433
7,8,2022,13239
8,9,2021,1810
9,10,2021,1341


### Algum IP específico?

In [170]:
ips = df.groupby(['ip']).count()
ips.rename(columns={'status': 'acessos'}, inplace=True)
ips = ips.acessos

In [171]:
ips.sort_values(ascending=False)

ip
200.129.47.254     40665
216.245.221.89     25274
191.5.210.64       12275
10.0.84.198         8228
172.18.0.1          8185
                   ...  
120.52.152.20          1
178.72.75.143          1
201.150.176.204        1
178.72.76.126          1
114.119.130.61         1
Name: acessos, Length: 7354, dtype: int64

### Padrões de recursos?

In [172]:
recursos_validos = ['.well-known', 'Autodiscover', 'avaliacao', 'cgs', 'chaves', 'cieco', 'cobras', 'computacao', 'cppgi', 'db', 'docs', 'eleicoes', 'evox', 'facd', 'hestia', 'index.php?s=', 'main', 'mongo', 'pesquisa', 'phpmyadmin', 'public', 'webapi', 'yoko']

In [173]:
dbrecurso = pd.read_csv('recurso.csv', sep=',')

In [174]:
qtdrecursos = dbrecurso.groupby(['status', 'recurso']).count()
qtdrecursos.rename(columns={'data1': 'acessos'}, inplace=True)
qtdrecursos = qtdrecursos.acessos

In [175]:
qtdrecursos.sort_values(ascending=False)

status  recurso            
200     computacao             222833
        pesquisa                76438
        avaliacao               55178
        main                    25427
        phpmyadmin              20995
                                ...  
404     premise                     1
        SQLiteManager-1.0.4         1
        dbmanager                   1
        SQLite-Manager              1
-       %2f                         1
Name: acessos, Length: 1801, dtype: int64

In [176]:
# Agrupar por recurso
df[df['recurso'].str.contains('evox')].groupby('status').count()

,data1,data2,ip,metodo,recurso
status,,,,,
200,1,1,1,1,1
404,29,29,29,29,29


### Quantos métodos diferentes ?

In [179]:
metodos = df.groupby(['status', 'metodo']).count()
metodos.rename(columns={'data1': 'acessos'}, inplace=True)
metodos = metodos.acessos
metodos = metodos.reset_index()

In [180]:
metodos.sort_values(by='acessos', ascending=False)

,status,metodo,acessos
7,200,GET,321667
9,200,POST,100149
8,200,HEAD,25371
11,404,GET,16461
12,404,HEAD,3288
15,404,POST,1735
10,404,CONNECT,1558
1,-,GET,1163
17,429,GET,858
5,-,POST,79


## Gráficos

In [181]:
import plotly.offline as py
import plotly.graph_objs as go

py.init_notebook_mode(connected=True)

### Acessos por data:

In [190]:
graf2021 = go.Scatter(x=['Setembro', 'Outubro', 'Dezembro'],
                    y=dateAcess['Acessos'][8:],
                    name='2021')

graf2022 = go.Scatter(x=['Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 'Julho', 'Agosto'],
                    y=dateAcess['Acessos'][0:8],
                    name='2022')

layout = go.Layout(title='Acessos por Data',
                   yaxis={'title':'Quantidade de Acessos'},
                   xaxis={'title': 'Mês de Acesso'})

fig = go.Figure(data=[graf2021, graf2022], layout=layout)

py.iplot(fig)

### Acessos por método:

In [185]:
grafMetodo = go.Bar(x = metodos['metodo'],
                  y = metodos['acessos'])

layout = go.Layout(title='Acessos por Métodos',
                   yaxis={'title':'Quantidade de Acessos'},
                   xaxis={'title': 'Métodos'})

fig = go.Figure(data=grafMetodo, layout=layout)

py.iplot(fig)

### Acessos por status/método:

In [186]:
metodosOK = metodos[metodos.status == '200']
metodosERROR = metodos[metodos.status >= '400']

In [187]:
grafico = [go.Bar(x = metodosOK['metodo'],
                   y = metodosOK['acessos'],
               name = 'Sucesso'),
           go.Bar(x = metodosERROR['metodo'],
                  y = metodosERROR['acessos'],
                  name = 'Erro')]

layout = go.Layout(title='Acessos por Status/Método',
                   yaxis={'title':'Método'},
                   xaxis={'title': 'Quantidade de Acessos'})

fig = go.Figure(data=grafico, layout=layout)

py.iplot(fig)